# Classification rules for covid dataset


## 1. Classification Rules Implementation

### 1.1. PRISM algorithm
In this lab we are going to implement the PRISM algorithm to extract the classification rules with the highest accuracy and coverage from the hospital patients dataset described in the class demo notebook.

Our algorithm extracts the rules ranked by the accuracy (from highest to lowest), and the ties are broken by choosing the rule with higher coverage. If both accuracy and coverage are the same - the condition is selected arbitrarily.

Your algorithm should have two additional optinal parameters:

the accuracy threshold - the number from 0 to 1 which specifies when you need to stop refining the rule. I.e. if you reached this accuracy threshold, you do not need to add more conditions to the rule's antedescent.
the coverage threshold - the absolute number of records covered by the rule. If the more precise rule covers less records than this threshold, the algorithm should stop refining this rule.
The output of the algorithm should be a decision list, where all the rules are presented ranked as above, including the rule itself, its accuracy, and its coverage.

Please keep your code clean, modular, and add explanations for each step.

Finally, apply the algorithm to the COVID-19 dataset to learn reliable rules which determine which symptoms/preexisting conditions and their combination lead to the deadly outcomes of the COVID-19 infection.

In your report (a separate markdown cell), summarize which rules did you learn from this dataset, and whether you were surpised by any of your discoveries.

In [1]:
import pandas as pd

Dividing set of observations `rows` based on the value `value` in column `column`.
It can split on numeric value (greater than or equal) or on categorical value (equal/not equal).

In [2]:
def split(rows, column, value):
    # define split function according to the value type
    split_function = None
    if isinstance(value, int) or isinstance(value, float):
        split_function = lambda row: row[column] >= value
    else:
        split_function = lambda row: row[column] == value

    # Divide the rows into two sets and return them
    set1 = [row for row in rows if split_function(row)]
    set2 = [row for row in rows if not split_function(row)]
    return (set1, set2)

To evaluate the quality of the split, we need to count the occurrence of each class label in the current subset (potential tree node).

In [3]:
def count_labels(rows):
    label_count = {}
    for row in rows:
        # The class label is in the last column
        label = row[- 1]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1
    return label_count

Different methods to evaluate the homogeneity of the target variable within a given subset generated by split.

* GINI:  probability that two items randomly selected fall into the same class. We use here GINI impurity, because we want to use the same code which chooses the best split based on the **minimum** of the score. GINI impurity = 1.0 - GINI.
* Entropy: the sum of $-p(x)\log(p(x))$ across all the different possible classes $x$.
* Variance: variance is used if the target variable can be considered a number. In this case we build a special type of decision tree - a **Regression Tree**.

In [ ]:
def learn_one_rule(data, accu_threshold, coverage_threshold):
    # Find class labels and attributes from data
    columns = list(data.columns)
    attributes = columns[:-1]
    class_labels = data[columns[-1]].unique()
    
    # Find the rule for the dataset for each current class label
    for clas in class_labels:
        cur_data = data[columns[-1]=clas]
        rule_lhs = np.array()
        rule = np.array(rule_lhs,clas)
        
        while accuracy(rule) > accu_threshold or cur_attr.length == 0:
            
            # For each attribute a not mentioned in the rule, and each attr value v, consider adding the condition a = v to rule_lhs
            cur_attr = attributes
            best_accur = 0
            best_pair = []
            for a in cur_attr:
                values = data[a].unique()
                for v in values:
                    rule[0].append([a,v])
                    
                    # select the rule of the attribute,value pair with best accuracy
                    if accuracy(rule) > best_accur:
                        best_accur = accuracy(rule)
                        best_pair = [a,v]
                    elif accuracy(rule) == best_accur:
                        cur_correct = data[]
                        correst_so_far = data[]

Now we have everything we need to build the decision tree from data.

The `buildtree` function below takes as an input all the rows of the dataset and returns a decision tree with the root  implemented as `DecisionNode` class.

The default `score_func` to test the quality of the split is entropy, but any other score can be applied by passing any of the scoring functions above. Remember that `variance` can only be applied if the target variable is numeric (Regression Tree).

To avoid overfitting, the algorithm uses additional stop criteria: if the improvement of the score is less than `min_improvement` or if the number of samples in every node is less than `min_samples`, then the splitting of samples stops. For the sake of the visual demo, there is also a parameter called `max_depth`, which controls the maximum depth of the tree. 

With the default values the algorithm will continue until the score of the split stops improving.

In [ ]:
def buildtree(rows, score_func=entropy, min_improvement=0, min_samples=0, max_depth=None, depth=0):
    if len(rows) == 0:
        return DecisionNode()
    # Compute overall score for the entire rows dataset
    current_score = score_func(rows)

    # Set up accumulator variables to track the best split criteria
    best_score = current_score
    best_criteria = None
    best_sets = None
    
    # Total number of features - except the last column where we store the class (target)
    column_count = len(rows[0]) - 1 
    for col in range(0, column_count):
        # Generate the list of unique values in
        # this column to split on them
        column_values = set()
        for row in rows:
            column_values.add(row[col])
            
        # Now try splitting the rows 
        # on each unique value in this column
        for value in column_values:
            (set1, set2) = split(rows, col, value)

            # Evaluate the quality of the split
            # p is the proportion of subset set1 
            p = float(len(set1)) / len(rows)
            split_score = p * score_func(set1) + (1-p) * score_func(set2)
            
            if split_score < best_score and \
                (len(set1) > min_samples and len(set2) > min_samples) and \
                (current_score - split_score) > min_improvement:
                best_score = split_score
                best_criteria = (col, value)
                best_sets = (set1, set2)

    # Create the sub branches
    if (current_score - best_score) > min_improvement and \
        (max_depth is None or depth < max_depth) :
        # print("Splitting on",best_criteria, " 2 sets:", len(best_sets[0]),len(best_sets[1]))
        true_branch = buildtree(best_sets[0], score_func, min_improvement, min_samples, max_depth, depth+1)
        false_branch = buildtree(best_sets[1], score_func, min_improvement, min_samples, max_depth, depth+1)
        return DecisionNode(col=best_criteria[0], value=best_criteria[1],
                            tb=true_branch, fb=false_branch)
    else: # Done splitting - summarize class labels in leaf nodes
        return DecisionNode(results=count_labels(rows))

## 2. Using learned tree for classification

### 2.1. Classifying fruits

First, build the tree from data.

In [ ]:
# fruits with their size and color
fruits = [
    [4, 'red', 'apple'],
    [4, 'green', 'apple'],
    [1.5, 'red', 'cherry'],
    [1, 'green', 'grape'],
    [5, 'red', 'apple'],
    [1.2, 'red', 'grape']
]

tree = buildtree(fruits)

The `prediction` function below outputs the label of the leaf node according to a prevalent class.
It cannot be used for regression trees - you need to implement your own function if you want to use this code for predicting a numeric target.

In [ ]:
def prediction(leaf_labels):
    total = 0
    result = {}
    for label, count in leaf_labels.items():
        total += count
        result[label] = count

    for label, val in result.items():
        result[label] = str(int(result[label]/total * 100))+"%"

    return result

## 3. Coronavirus risk factors

As discussed in the lecture, decision trees can be used not only for classification, but also to find out which atttributes are most important in classifying the record into a specific class. In this part we want to find out which symptoms/chronic conditions contribute most to the deadly outcome from catching COVID-19.

Tis Mexican dataset which contains the information from the Statistical Yearbooks of Morbidity 2015-2017 (as well as the information regarding cases associated with COVID-19) was found on [kaggle](https://www.kaggle.com/tanmoyx/covid19-patient-precondition-dataset).

Download the preprocessed dataset which contains only patients that tested positive for COVID-19 and with symptom atributes converted to categorical: [link](https://docs.google.com/spreadsheets/d/1EPewR1KdT8mszXJMuqELRTHAVIFHEJw9VwLsb9whKPI/edit?usp=sharing).

In this dataset we have the following attributes:
1. sex: 1 -woman, 2-man
2. age: numeric
3. diabetes: yes/no
4. copd (chronic obstructive pulmonary disease): yes/no
5. asthma: yes/no
6. imm_supr (suppressed immune system): yes/no
7. hypertension: yes/no
8. cardiovascular: yes/no
9. renal_chronic: yes/no
10. tobacco: yes/no	
11. outcome: alive/dead

In [ ]:
data_file = "../../data_ml_2020/covid_categorical_good.csv"

In [ ]:
import pandas as pd
data = pd.read_csv(data_file)
data = data.dropna(how="any")
data.columns

In [ ]:
data_rows = data.to_numpy().tolist()
len(data_rows)

In [ ]:
columns_list = data.columns.to_numpy().tolist()
print(columns_list)

### 3.1. Using custom decision tree algorithm

In [ ]:
tree = buildtree(data_rows, score_func=entropy, min_improvement=0, min_samples=0, max_depth=7)

In [ ]:
print_tree(tree, '', columns_list)

### 3.2. Using sklearn

The decision tree algorithm in sklearn library is not implemented very well. It requires all the attributes to be numeric - while decision trees work best with the categorical attributes. The dataset for this part contains all numeric attributes and can be found [here](https://docs.google.com/spreadsheets/d/1FHTP2RtclUg05GztDW-diMbajMAynPnECW8hyjNLUys/edit?usp=sharing).

In this dataset the binary yes/no attributes are presented as:
* YES: 1
* NO: 2 

In [ ]:
data_file =  "../../data_ml_2020/covid_numeric.csv"

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import tree

In [ ]:
data_num = pd.read_csv(data_file)

Remove rows with missing values

In [ ]:
for k in ['sex', 'diabetes', 'copd', 'asthma', 'imm_supr', 'hypertension',
       'cardiovascular', 'obesity', 'renal_chronic', 'tobacco']:
    data_num[k].replace({97: np.nan, 98: np.nan, 99: np.nan}, inplace=True)


data_num = data_num.dropna()
len(data_num)

We want to see if there is a correlation between any of the attributes and the outcome. We will replace the "alive" outcome with number 0, and "dead" with number 2.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

data_num_ = data_num.copy()
conditions = [ data_num_['outcome'].eq("alive"), data_num_['outcome'].eq('dead')]
choices = [0,2]

data_num_['outcome'] = np.select(conditions, choices, default=np.nan)

corr = data_num_.corr()
sns.heatmap(corr)

Divide dataset into features and class label.

In [ ]:
X = data_num.loc[:, data_num.columns != 'outcome']
print(X.columns)

Y = data_num.loc[:, data_num.columns == 'outcome']
print(Y.columns)

Split dataset intro training and testing.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)

Different parameters can be specified to build the model:

`model = tree.DecisionTreeClassifier(
        criterion='entropy', 
        max_depth=None, 
        min_samples_split=2, 
        min_samples_leaf=1, 
        max_features=None, 
        random_state=None, 
        min_density=None, 
        compute_importances=None, 
        max_leaf_nodes=None)`

In [ ]:
model = tree.DecisionTreeClassifier(criterion='entropy', max_depth = 7)
model.fit(X_train,Y_train)

In [ ]:
columns_list = data.columns.to_numpy().tolist()
from sklearn.tree import export_text
r = export_text(model, feature_names=columns_list[:-1])
print(r)

What are the most important comorbidity factors? Hard to tell. 
We will try to discover them more efficiently in the next lab using classification rules.

Copyright &copy; 2020 Marina Barsky. All rights reserved.